In [1]:
import pandas as pd
import numpy as np
import re
import datetime
import requests
import bs4
import json

pd.options.display.max_rows = 500
pd.options.display.max_columns = 500
pd.set_option('display.max_colwidth', -1)


In [2]:
url = "https://berlin.craigslist.org/d/housing/search/hhh"

In [3]:
r = requests.get(url)

In [4]:
soup = bs4.BeautifulSoup(r.content, "html.parser")

In [5]:
soup.body

<body class="search">
<script type="text/javascript"><!--
    function C(k){return(document.cookie.match('(^|; )'+k+'=([^;]*)')||0)[2]}
    var pagetype, pagemode;
    (function(){
        var h = document.documentElement;
        h.className = h.className.replace('no-js', 'js');
        var b = document.body;
        var bodyClassList = b.className.split(/\s+/);;
        pagetype = bodyClassList[0]; // dangerous assumption
        var fmt = C('cl_fmt');
        if ( fmt === 'regular' || fmt === 'mobile' ) {
            pagemode = fmt;
        } else if (screen.width <= 480) {
            pagemode = 'mobile';
        } else {
            pagemode = 'regular';
        }
        pagemode = pagemode === 'mobile' ? 'mobile' : 'desktop';
        bodyClassList.push(pagemode);
        if (C('hidesearch') === '1' && pagemode !== 'mobile') {
            bodyClassList.push('hide-search');
        }
        var width = window.innerWidth || document.documentElement.clientWidth;
        if (width >

## Find information for first appartement

In [6]:
app1_on_website = soup.find("p", class_="result-info")

In [7]:
app1_posted = app1_on_website.find("time", class_="result-date").contents[0]

In [8]:
app1_header= str(app1_on_website.find("a", class_="result-title hdrlnk").contents[0])

In [9]:
app1_price = str(app1_on_website.find("span", class_="result-price").contents[0])

In [10]:
app1_hood = str(app1_on_website.find("span", class_="result-hood").contents[0])

## Find information for first appartement with appartement_list

Find appartement list

In [11]:
appartement_list = soup.find("ul", class_="rows").find_all("li", class_="result-row")

Find picture url

In [61]:
appartement_list[1].find("div", class_="swipe-wrap")

In [63]:
appartement_list[1]

<li class="result-row" data-pid="7075736469" data-repost-of="6897078903">
<a class="result-image gallery" data-ids="1:00l0l_a46im4xHGL8,1:01111_cqlde2K9ZIL,1:00j0j_cB2P93DTwD2,1:00505_h0DKsGKbGLT,1:00g0g_fGabFo91fDC,1:00E0E_W22JMZhTfo,1:00K0K_7blW7bJS032,1:00m0m_cMeaEi6fyBe,1:00707_1YnHjngsmdQ,1:00m0m_2NEbLMDsly3,1:00E0E_7Y2th7ZL3Gr" href="https://berlin.craigslist.org/apa/d/1-bd-in-the-middle-of-berlin-prenzlauer/7075736469.html">
<span class="result-price">€500</span>
</a>
<p class="result-info">
<span class="icon icon-star" role="button">
<span class="screen-reader-text">post zu favoriten</span>
</span>
<time class="result-date" datetime="2020-02-15 15:52" title="Sa 15 Feb 15:52:03">Feb 15</time>
<a class="result-title hdrlnk" data-id="7075736469" href="https://berlin.craigslist.org/apa/d/1-bd-in-the-middle-of-berlin-prenzlauer/7075736469.html">1 BD In the middle of Berlin-Prenzlauer Berg up to 6 guests</a>
<span class="result-meta">
<span class="result-price">€500</span>
<span clas

In [56]:
app1_picture = appartement_list[0].find(".jpg").contents[0]

AttributeError: 'NoneType' object has no attribute 'contents'

In [44]:
app1_picture

'Feb 15'

In [13]:
len(appartement_list)

120

In [14]:
app1_posted = str(appartement_list[0].find("time", class_="result-date").contents[0])
app1_header = str(appartement_list[0].find("a", class_="result-title hdrlnk").contents[0])
app1_price = str(appartement_list[0].find("span", class_="result-price").contents[0])

app1_bedrooms_and_size = str(appartement_list[4].find("span", class_="housing").contents[0]) if appartement_list[4].find("span", class_="housing") != None else ""
try:
    app1_bedrooms_only = re.search('(.+?)br', app1_bedrooms_and_size).group(1)
except AttributeError:
    app1_bedrooms_only = '' #apply your error handling
app1_bedrooms_only = app1_bedrooms_only.replace(" ", "")

app1_hood = str(appartement_list[0].find("span", class_="result-hood").contents[0])

extract bedrooms from bedrooms and size

In [15]:
app1_bedrooms_only

'1'

In [16]:
app1_bedrooms_and_size = str(appartement_list[4].find("span", class_="housing").contents[0]) if appartement_list[4].find("span", class_="housing") != None else ""


In [17]:
app1_bedrooms_and_size

'\n                    1br -\n                '

In [18]:
text = 'gfgfdAAA1234ZZZuijjk'
text1 = app1_bedrooms_and_size
try:
    found = re.search('(.+?)br', text1).group(1)
except AttributeError:
    # AAA, ZZZ not found in the original string
    found = '' #apply your error handling


In [19]:
found.replace(" ", "")

'1'

Extract size from bedrooms and size

In [20]:
app1_bedrooms_and_size

'\n                    1br -\n                '

In [21]:
text = 'gfgfdAAA1234ZZZuijjk'
text1 = app1_bedrooms_and_size
try:
    found = re.search('(.+?)m', text1).group(1)
except AttributeError:
    # AAA, ZZZ not found in the original string
    found = '' #apply your error handling


In [22]:
found

''

## Combine information in a dataframe

In [23]:
column_list = ["posted", "header", "price", "bedrooms", "size", "location", "picture"]

In [24]:
df_app = pd.DataFrame(columns = column_list)

In [25]:
df_app

,posted,header,price,bedrooms,size,location,picture


In [26]:
for appartement in appartement_list:
    app_posted = str(appartement.find("time", class_="result-date").contents[0])
    app_header = str(appartement.find("a", class_="result-title hdrlnk").contents[0])
    app_price = str(appartement.find("span", class_="result-price").contents[0])
    
    app_bedrooms_and_size = str(appartement.find("span", class_="housing").contents[0]) if appartement.find("span", class_="housing") != None else ""
    
    # Number of bedrooms:
    try:
        app_bedrooms = re.search('(.+?)br', app_bedrooms_and_size).group(1)
    except AttributeError:
        app_bedrooms = ''
    app_bedrooms = app_bedrooms.replace(" ", "")
    
    # Appartement size:
    try:
        app_size = re.search('(.+?)m', app_bedrooms_and_size).group(1)
    except AttributeError:
        app_size = ''
    app_size = app_size.replace(" ", "")
    
    
    app_hood = str(appartement.find("span", class_="result-hood").contents[0]) if appartement.find("span", class_="result-hood") != None else ""
    app_picture = "https://images.craigslist.org/00404_hvNQ0G2OSeq_600x450.jpg"
    
    app_information = [app_posted, app_header, app_price, app_bedrooms, app_size, app_hood, app_picture]
    
    df_app.loc[len(df_app)] = app_information

In [27]:
#df_app.head(5)

In [28]:
df_app["price"] = df_app["price"].str.replace("€", "")

In [29]:
df_app["location"] = df_app["location"].str.replace("(", "").str.replace(")", "")

In [30]:
df_app

,posted,header,price,bedrooms,size,location,picture
0,Feb 15,"Berlinale, two-room suite, own entrance + bike",0,,45,Charlottenburg- Berlin,https://images.craigslist.org/00404_hvNQ0G2OSeq_600x450.jpg
1,Feb 15,1 BD In the middle of Berlin-Prenzlauer Berg up to 6 guests,500,1,54,Prenzlauer Berg,https://images.craigslist.org/00404_hvNQ0G2OSeq_600x450.jpg
2,Feb 15,"Two room suite with own entrance, furnished",1000,,,Charlottenburg- Berlin,https://images.craigslist.org/00404_hvNQ0G2OSeq_600x450.jpg
3,Feb 15,1X1 Sun-flooded luxury studio in prime location Prenzlauer Berg,300,1,52,"Kollwitzstraße, Berlin, Germany",https://images.craigslist.org/00404_hvNQ0G2OSeq_600x450.jpg
4,Feb 15,%%%Fabulous! Remodeled One Bedroom! Make it Yours Today!,350,1,,"Reichenberger Str., Friedrichshain-Kreuzberg, 10999 Berlin,",https://images.craigslist.org/00404_hvNQ0G2OSeq_600x450.jpg
5,Feb 15,---Fabulous! Remodeled One Bedroom! Make it Yours Today!,350,1,,Friedrichshain-Kreuzberg,https://images.craigslist.org/00404_hvNQ0G2OSeq_600x450.jpg
6,Feb 15,===Fabulous! Remodeled One Bedroom! Make it Yours Today!,350,1,,Reichenberger Str.,https://images.craigslist.org/00404_hvNQ0G2OSeq_600x450.jpg
7,Feb 15,Rent your dream 1 bed 1 bath today! Newly renovated units. 55 SqFt,700,1,55,Wartenburgstraße Kreuzberg,https://images.craigslist.org/00404_hvNQ0G2OSeq_600x450.jpg
8,Feb 15,Exceptional 2 Bedroom Maisonette Rooftop Views,800,2,112,Mitte,https://images.craigslist.org/00404_hvNQ0G2OSeq_600x450.jpg
9,Feb 15,Exceptional 2 Bedroom Maisonette Rooftop Views,800,2,112,Mitte,https://images.craigslist.org/00404_hvNQ0G2OSeq_600x450.jpg


In [31]:
df_app = df_app.replace(r'^\s*$', 0, regex=True)

Convert price, bedrooms and size to int

In [32]:
df_app

,posted,header,price,bedrooms,size,location,picture
0,Feb 15,"Berlinale, two-room suite, own entrance + bike",0,0,45,Charlottenburg- Berlin,https://images.craigslist.org/00404_hvNQ0G2OSeq_600x450.jpg
1,Feb 15,1 BD In the middle of Berlin-Prenzlauer Berg up to 6 guests,500,1,54,Prenzlauer Berg,https://images.craigslist.org/00404_hvNQ0G2OSeq_600x450.jpg
2,Feb 15,"Two room suite with own entrance, furnished",1000,0,0,Charlottenburg- Berlin,https://images.craigslist.org/00404_hvNQ0G2OSeq_600x450.jpg
3,Feb 15,1X1 Sun-flooded luxury studio in prime location Prenzlauer Berg,300,1,52,"Kollwitzstraße, Berlin, Germany",https://images.craigslist.org/00404_hvNQ0G2OSeq_600x450.jpg
4,Feb 15,%%%Fabulous! Remodeled One Bedroom! Make it Yours Today!,350,1,0,"Reichenberger Str., Friedrichshain-Kreuzberg, 10999 Berlin,",https://images.craigslist.org/00404_hvNQ0G2OSeq_600x450.jpg
5,Feb 15,---Fabulous! Remodeled One Bedroom! Make it Yours Today!,350,1,0,Friedrichshain-Kreuzberg,https://images.craigslist.org/00404_hvNQ0G2OSeq_600x450.jpg
6,Feb 15,===Fabulous! Remodeled One Bedroom! Make it Yours Today!,350,1,0,Reichenberger Str.,https://images.craigslist.org/00404_hvNQ0G2OSeq_600x450.jpg
7,Feb 15,Rent your dream 1 bed 1 bath today! Newly renovated units. 55 SqFt,700,1,55,Wartenburgstraße Kreuzberg,https://images.craigslist.org/00404_hvNQ0G2OSeq_600x450.jpg
8,Feb 15,Exceptional 2 Bedroom Maisonette Rooftop Views,800,2,112,Mitte,https://images.craigslist.org/00404_hvNQ0G2OSeq_600x450.jpg
9,Feb 15,Exceptional 2 Bedroom Maisonette Rooftop Views,800,2,112,Mitte,https://images.craigslist.org/00404_hvNQ0G2OSeq_600x450.jpg


In [33]:
convert_dict = {"price": int,
                "bedrooms": int,
                "size": int
               }

In [34]:
df_app = df_app.astype(convert_dict)

In [35]:
df_app.dtypes

posted      object
header      object
price       int32 
bedrooms    int32 
size        int32 
location    object
picture     object
dtype: object

In [36]:
json_test = df_app.iloc[0:5].to_json(orient="index")

In [37]:
json_test

'{"0":{"posted":"Feb 15","header":"Berlinale,  two-room suite, own entrance + bike","price":0,"bedrooms":0,"size":45,"location":" Charlottenburg- Berlin","picture":"https:\\/\\/images.craigslist.org\\/00404_hvNQ0G2OSeq_600x450.jpg"},"1":{"posted":"Feb 15","header":"1 BD In the middle of Berlin-Prenzlauer Berg up to 6 guests","price":500,"bedrooms":1,"size":54,"location":" Prenzlauer Berg","picture":"https:\\/\\/images.craigslist.org\\/00404_hvNQ0G2OSeq_600x450.jpg"},"2":{"posted":"Feb 15","header":"Two room suite with own entrance, furnished","price":1000,"bedrooms":0,"size":0,"location":" Charlottenburg- Berlin","picture":"https:\\/\\/images.craigslist.org\\/00404_hvNQ0G2OSeq_600x450.jpg"},"3":{"posted":"Feb 15","header":"1X1 Sun-flooded luxury studio in prime location Prenzlauer Berg","price":300,"bedrooms":1,"size":52,"location":" Kollwitzstra\\u00dfe, Berlin, Germany","picture":"https:\\/\\/images.craigslist.org\\/00404_hvNQ0G2OSeq_600x450.jpg"},"4":{"posted":"Feb 15","header":"%%%

In [38]:
json_appartement_data = df_app.to_json(orient="records")

In [ ]:
appartement_data_dict = {"app": json_appartement_data}

In [ ]:
appartement_data_dict["app"]

In [ ]:
#with open('appartement_data_scraped.txt', 'w') as outfile:
#    json.dump(json_appartement_data, outfile)

In [ ]:
with open('appartement_data_scraped.json', 'w') as outfile:
    json.dump(appartement_data_dict, outfile)